# Temporal Analysis of Temperature Trends

Introduction to Notebook

## Set Up

In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as stats
from scipy import signal
import pymannkendall as mk
import sys
import os

# Update paths to get source code from notebook_utils
curr_dir = os.path.dirname(os.path.abspath('notebooks'))
proj_dir = os.path.dirname(curr_dir)
src_path = os.path.join(proj_dir, 'src')
sys.path.append(src_path)

from notebook_utils.preprocessing import *

# Create combined dataframe from ghcn_cleaned files
dfs = combine_files_to_dfs('../data/processed/ghcn_cleaned')
CA_stations = pd.concat(dfs, ignore_index=True)

Processed file: CA_2003_clean.csv
Processed file: CA_2004_clean.csv
Processed file: CA_2005_clean.csv
Processed file: CA_2006_clean.csv
Processed file: CA_2007_clean.csv
Processed file: CA_2008_clean.csv
Processed file: CA_2009_clean.csv
Processed file: CA_2010_clean.csv
Processed file: CA_2011_clean.csv
Processed file: CA_2012_clean.csv
Processed file: CA_2013_clean.csv
Processed file: CA_2014_clean.csv
Processed file: CA_2015_clean.csv
Processed file: CA_2016_clean.csv
Processed file: CA_2017_clean.csv
Processed file: CA_2018_clean.csv
Processed file: CA_2019_clean.csv
Processed file: CA_2020_clean.csv
Processed file: CA_2021_clean.csv
Processed file: CA_2022_clean.csv
Processed file: CA_2023_clean.csv


# Trend Analysis

Analyze trends in temperature data over time.

### (Maybe) Linear Regression

### (Maybe) Mann-Kendall Trend Test

# Seasonal Variations

Examine seasonal patterns and changes in temperature across different years.

# Summarize Findings